# Imports

In [1]:
import numpy as np 
import cv2 
import matplotlib.pyplot as plt
import PIL.Image as Image
import gym
import random

from gym import Env, spaces
import time

font = cv2.FONT_HERSHEY_COMPLEX_SMALL 

# Vehicle Area Model

In [2]:
class VehicleArea(Env):
    def __init__(self, train = True):
        super(VehicleArea, self).__init__()
        self.canvas_shape = (900, 900, 3)

        # Define action and observation space
        self.observation_shape = (45, 45, 3)

        self.observation_space = spaces.Dict({"position":spaces.Box(low = np.zeros(self.observation_shape), 
                                                        high = np.ones(self.observation_shape),
                                                        dtype = np.float32), 
                                            "orientation":spaces.Box(low = np.array([-np.pi]),
                                            high = np.array([np.pi]),
                                            dtype = np.float32)})

        self.action_space = spaces.Dict({"linear_velocity":spaces.Box(low = np.array([-100]),
                                            high = np.array([1000]),
                                            dtype = np.float32),
                                        "angular_velocity":spaces.Box(low = np.array([-2]),
                                        high = np.array([2]), 
                                        dtype = np.float32)})

        self.delta_t = 0.1

        self.train = train
        # Maximum fuel vehicle can take at once
        self.max_fuel = 1000

        # Define elements present inside the environment
        self.elements = []
        # Define visted cells within the environment
        self.visited = []

        # Permitted area of the environment
        self.y_min = 0
        self.x_min = 0
        self.y_max = self.canvas_shape[0] 
        self.x_max = self.canvas_shape[1]
        self.theta_min = -np.pi
        self.theta_max = np.pi
                                
        # Create a canvas to render the environment images upon 
        self.canvas = np.ones(self.canvas_shape, dtype=np.uint8) * 255
    
    def draw_elements_on_canvas(self):
        # Init the canvas 
        self.canvas = np.ones(self.canvas_shape, dtype=np.uint8) * 255

        # Draw the elements on canvas
        for elem in self.visited:
            elem_shape = elem.icon.shape
            x,y = elem.x, elem.y
            self.canvas[y : y + elem_shape[1], x:x + elem_shape[0]] = elem.icon

        text = 'Fuel Left: {} | Rewards: {}'.format(self.fuel_left, self.ep_return)

        # Draw the vehicle on canvas
        self.canvas[self.vehicle.y : self.vehicle.y + self.vehicle.icon.shape[1], self.vehicle.x:self.vehicle.x + self.vehicle.icon.shape[0]] = self.vehicle.icon
        

        # Put the info on canvas 
        self.canvas = cv2.putText(self.canvas, text, (10,20), font,  
                   0.8, (0,0,0), 1, cv2.LINE_AA)

    
    def reset(self):
        # Reset the fuel consumed
        self.fuel_left = self.max_fuel
    
        # Reset the reward
        self.ep_return  = 0

        # Number of visited cells
        self.visited_count = 0
    
        # Determine a place to intialise the vehicle in the environment (top left corner)
        x = random.randrange(int(0), int(self.observation_shape[0]), 90)
        y = random.randrange(int(0), int(self.observation_shape[1]), 90)
        theta = random.uniform(self.theta_min, self.theta_max)
        
        # Intialise the vehicle
        self.vehicle = Vehicle("vehicle", self.x_max, self.x_min, self.y_max, self.y_min, self.theta_max, self.theta_min)
        self.vehicle.set_position(x,y,theta)
    
        # Intialise the elements 
        self.visited = []
    
        # Intialise the elements 
        self.visited = [] # self.vehicle

        spawned_visit = Visited("visit_{}".format(self.visited_count), self.x_max, self.x_min, self.y_max, self.y_min)
        self.visited_count += 1
        spawned_visit.set_position(self.vehicle.x, self.vehicle.y, 0)
        self.visited.append(spawned_visit)
    
        # Reset the Canvas 
        self.canvas = np.ones(self.canvas_shape) * 255

        # Draw elements on the canvas
        self.draw_elements_on_canvas()

        # return the observation
        return cv2.resize(self.canvas, (45,45))
        



    def render(self, mode = "human"):
        assert mode in ["human", "rgb_array"], "Invalid mode, must be either \"human\" or \"rgb_array\""
        if mode == "human":
            cv2.imshow("Game", self.canvas)
            cv2.waitKey(10)
        
        elif mode == "rgb_array":
            return self.canvas
        
    def close(self):
        cv2.destroyAllWindows()

    def step(self, action):
        # Flag that marks the termination of an episode
        done = False
        # Reward for the current step
        reward = 0
        # Assert that it is a valid action 
        assert self.action_space.contains(action), "Invalid Action"

        # Decrease the fuel counter 
        self.fuel_left -= 1 

   

        # apply the action to the vehicle
        if action["linear_velocity"] != 0:
            self.vehicle.move(action["linear_velocity"]*self.delta_t) # *np.cos(self.vehicle.orientation))
        if action["angular_velocity"] != 0:
            self.vehicle.rotate(action["angular_velocity"])

        
        # Check if the vehicle is visiting a new cell
        previously_visited = False
        for cell in self.visited:
            if self.vehicle.x == cell.x and self.vehicle.y == cell.y:
                previously_visited = True
                break

        if not previously_visited:                               
            spawned_visit = Visited("visit_{}".format(self.visited_count), self.x_max, self.x_min, self.y_max, self.y_min)
            self.visited_count += 1
            spawned_visit.set_position(self.vehicle.x, self.vehicle.y, 0)
            self.visited.append(spawned_visit)
       

            # Reward for executing a step towards a new cell
            reward += 25
        else:
            # Reward for executing a step towards a previously visited cell 
            reward += -5
        
        

        # Draw elements on the canvas
        self.draw_elements_on_canvas()

        # If out of fuel, end the episode.
        if self.fuel_left == 0:
            done = True
            reward += -30000 # Penalty for running out of fuel before reaching the goal

        #if self.visited_count >= 90: 
        #    done = True
        #    reward += 1000000 # Reward for reaching the goal

#
        # Increment the episodic return
        self.ep_return += reward

        # Draw elements on the canvas
        self.draw_elements_on_canvas()

        # If out of fuel, end the episode.
        if self.fuel_left == 0:
            done = True

        return cv2.resize(self.canvas, (45,45)), reward, done, {}
    
    


        

In [3]:
class Point(object):
    def __init__(self, name, x_max, x_min, y_max, y_min, theta_max, theta_min):
        self.x = 0
        self.y = 0
        self.orientation = 0
        self.x_min = x_min
        self.x_max = x_max
        self.y_min = y_min
        self.y_max = y_max
        self.theta_min = theta_min
        self.theta_max = theta_max
        self.name = name
    
    def set_position(self, x, y, theta):
        self.x = self.clamp(x, self.x_min, self.x_max - self.icon_w)
        self.y = self.clamp(y, self.y_min, self.y_max - self.icon_h)
        self.orientation = self.clamp(theta, self.theta_min, self.theta_max)
    
    def get_position(self):
        return (self.x, self.y)
    
    def move(self, distance):
        del_x = distance * np.cos(self.orientation)
        del_y = distance * np.sin(self.orientation)
        self.x += int(del_x)
        self.y += int(del_y)
        
        self.x = self.clamp(self.x, self.x_min, self.x_max - self.icon_w)
        self.y = self.clamp(self.y, self.y_min, self.y_max - self.icon_h)
    
    def rotate(self, theta):
        self.orientation += theta
        self.orientation = self.clamp(self.orientation, self.theta_min, self.theta_max)

    def clamp(self, n, minn, maxn):
        return max(min(maxn, n), minn)


In [4]:
class Vehicle(Point):
    def __init__(self, name, x_max, x_min, y_max, y_min, theta_max, theta_min):
        super(Vehicle, self).__init__(name, x_max, x_min, y_max, y_min, theta_max, theta_min)
        self.icon = cv2.imread("data/vehicle_ed.png") #/ 255.0 # , cv2.IMREAD_GRAYSCALE
        self.icon_w = 90
        self.icon_h = 90
        self.icon = cv2.resize(self.icon, (self.icon_h, self.icon_w))
        
        self.angular_velocity = 0
        self.linear_velocity = 0

class Visited(Point):
    def __init__(self, name, x_max, x_min, y_max, y_min, theta_max=0, theta_min=0):
        super(Visited, self).__init__(name, x_max, x_min, y_max, y_min, 0, 0)
        self.icon_w = 90
        self.icon_h = 90
        self.icon = 125*np.ones((self.icon_w, self.icon_h, 3), dtype=np.uint8)
        self.icon[:,:,1] = 255
        self.icon[::20,::20,:] = 0
        self.icon = cv2.resize(self.icon, (self.icon_h, self.icon_w))

In [5]:

from IPython import display

env = VehicleArea()
obs = env.reset()


while True:
    # Take a random action
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)
    
    # Render the game
    env.render()
    
    if done == True:
        break

env.close()

/home/nav-shaza/Desktop/currentProjects/cover_bot/.venv/lib/python3.8/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))
